# Setup

In [1]:
OPENSEARCH_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

OPENSEARCH_DASHBOARD_PORT = 5601

OPENSEARCH_CLUSTER_NAME = "opensearch-cluster.mavasbel.vpn.itam.mx"
OPENSEARCH_TOTAL_NODES = 3

OPENSEARCH_NODE_IPS = ["10.15.20.2"] * OPENSEARCH_TOTAL_NODES
OPENSEARCH_NODE_NAMES = [
    f"opensearch-node-{i+1}" for i in range(OPENSEARCH_TOTAL_NODES)
]
OPENSEARCH_NODE_REST_API_PORTS = [9200 + (i + 1) for i in range(OPENSEARCH_TOTAL_NODES)]
OPENSEARCH_NODE_PERF_ANA_PORTS = [
    16280 + (i + 1) for i in range(OPENSEARCH_TOTAL_NODES)
]
OPENSEARCH_NODE_HOSTNAMES = [
    f"{OPENSEARCH_NODE_NAMES[j]}.mavasbel.vpn.itam.mx"
    for j in range(OPENSEARCH_TOTAL_NODES)
]
OPENSEARCH_NODE_HTTP_HOSTNAMES = [
    f"https://{OPENSEARCH_NODE_HOSTNAMES[j]}:{OPENSEARCH_NODE_REST_API_PORTS[j]}"
    for j in range(OPENSEARCH_TOTAL_NODES)
]

OPENSEARCH_WORKDIR = "/usr/share/opensearch/data"

OPENSEARCH_INITIAL_ADMIN_PASSWORD = "OpenSearchP455"
# OPENSEARCH_DEFAULT_PASSWORD = "opensearch"
# OPENSEARCH_INIT_USER = "opensearch"
# OPENSEARCH_INIT_PASSWORD = "opensearch"

In [2]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)

# Stop opensearch-cluster.docker-compose.yml

In [3]:
!docker compose -f opensearch-cluster.docker-compose.yml down -v

time="2026-01-10T22:04:56-06:00" level=warning msg="The \"OPENSEARCH_INITIAL_ADMIN_PASSWORD\" variable is not set. Defaulting to a blank string."
time="2026-01-10T22:04:56-06:00" level=warning msg="The \"OPENSEARCH_INITIAL_ADMIN_PASSWORD\" variable is not set. Defaulting to a blank string."
time="2026-01-10T22:04:56-06:00" level=warning msg="The \"OPENSEARCH_INITIAL_ADMIN_PASSWORD\" variable is not set. Defaulting to a blank string."
 Container opensearch-dashboards  Stopping
 Container opensearch-dashboards  Stopped
 Container opensearch-dashboards  Removing
 Container opensearch-dashboards  Removed
 Container opensearch-node-3  Stopping
 Container opensearch-node-3  Stopped
 Container opensearch-node-3  Removing
 Container opensearch-node-3  Removed
 Container opensearch-node-2  Stopping
 Container opensearch-node-2  Stopped
 Container opensearch-node-2  Removing
 Container opensearch-node-2  Removed
 Container opensearch-node-1  Stopping
 Container opensearch-node-1  Stopped
 Contai

In [4]:
import shutil

if OPENSEARCH_START_FROM_SCRATCH:
    shutil.rmtree(DOCKER_MOUNTDIR, ignore_errors=True)
    Path(DOCKER_MOUNTDIR).mkdir(parents=True, exist_ok=True)

# Start opensearch-cluster.docker-compose.yml

In [3]:
import os
import yaml
from IPython.display import Markdown, display

node_cpus = "2.0"
node_memory = "2G"
node_start_heap = "1G"
node_max_heap = "1G"

opensearch_compose_dict = {
    "name": "opensearch-cluster",
    "services": {},
    "networks": {"opensearch-cluster": {"driver": "bridge"}},
}

for i in range(OPENSEARCH_TOTAL_NODES):

    node_mount_dir = os.path.join(
        DOCKER_MOUNTDIR,
        OPENSEARCH_NODE_NAMES[i],
    )
    Path(node_mount_dir).mkdir(parents=True, exist_ok=True)
    with open(
        os.path.join(node_mount_dir, "performance-analyzer.properties"), "w"
    ) as f:
        f.write(
            f"""# Performance Analyzer Configuration for {OPENSEARCH_NODE_NAMES[i]}
webservice-bind-host = 0.0.0.0
webservice-listener-port = {OPENSEARCH_NODE_PERF_ANA_PORTS[i]}
metrics-location = /dev/shm/performanceanalyzer/
cleanup-metrics-db-files = true
https-enabled = false
"""
        )

    opensearch_compose_dict["services"][OPENSEARCH_NODE_NAMES[i]] = {
        "image": "opensearchproject/opensearch:3.4.0",
        "container_name": OPENSEARCH_NODE_NAMES[i],
        "environment": [
            f"cluster.name={OPENSEARCH_CLUSTER_NAME}",
            f"node.name={OPENSEARCH_NODE_NAMES[i]}",
            f"discovery.seed_hosts={','.join(OPENSEARCH_NODE_NAMES)}",
            f"cluster.initial_cluster_manager_nodes={','.join(OPENSEARCH_NODE_NAMES)}",
            f"http.port={OPENSEARCH_NODE_REST_API_PORTS[i]}",
            "bootstrap.memory_lock=true",
            f"OPENSEARCH_JAVA_OPTS=-Xms{node_start_heap} -Xmx{node_max_heap}",
            f"OPENSEARCH_INITIAL_ADMIN_PASSWORD={OPENSEARCH_INITIAL_ADMIN_PASSWORD}",
            f'OPENSEARCH_HOSTS=["{'","'.join(OPENSEARCH_NODE_HTTP_HOSTNAMES)}"]',
        ],
        "ulimits": {
            "memlock": {"soft": -1, "hard": -1},
            "nofile": {"soft": 65536, "hard": 65536},
        },
        "volumes": [
            f"{os.path.join(node_mount_dir, "data")}:{OPENSEARCH_WORKDIR}",
            f"{os.path.join(node_mount_dir, "performance-analyzer.properties")}:/usr/share/opensearch/config/opensearch-performance-analyzer/performance-analyzer.properties",
        ],
        "networks": ["opensearch-cluster"],
        "hostname": OPENSEARCH_NODE_HOSTNAMES[i],
        "ports": [
            f"{OPENSEARCH_NODE_REST_API_PORTS[i]}:{OPENSEARCH_NODE_REST_API_PORTS[i]}",
            f"{OPENSEARCH_NODE_PERF_ANA_PORTS[i]}:{OPENSEARCH_NODE_PERF_ANA_PORTS[i]}",
        ],
        "extra_hosts": [
            f"{DOCKER_INTERNAL_HOST}:host-gateway",
        ],
        "dns": DOCKER_DNS,
        "deploy": {"resources": {"limits": {"cpus": node_cpus, "memory": node_memory}}},
        "depends_on": {
            OPENSEARCH_NODE_NAMES[j]: {"condition": "service_started"}
            for j in range(0, i)
        },
        "healthcheck": {
            "test": [
                "CMD-SHELL",
                f"curl -s -k -u admin:${{OPENSEARCH_INITIAL_ADMIN_PASSWORD}} "
                f'https://localhost:{OPENSEARCH_NODE_REST_API_PORTS[i]}/_cluster/health | grep -qv \'"status":"red"\'',
            ],
            "interval": "10s",
            "timeout": "10s",
            "retries": 10,
            "start_period": "30s",
        },
    }

opensearch_compose_dict["services"]["opensearch-dashboards"] = {
    "image": "opensearchproject/opensearch-dashboards:3.4.0",
    "container_name": "opensearch-dashboards",
    "environment": [
        f'OPENSEARCH_HOSTS=["{'","'.join(OPENSEARCH_NODE_HTTP_HOSTNAMES)}"]',
        f"OPENSEARCH_JAVA_OPTS=-Xms{node_start_heap} -Xmx{node_max_heap}",
    ],
    "networks": ["opensearch-cluster"],
    "ports": [
        f"{OPENSEARCH_DASHBOARD_PORT}:{OPENSEARCH_DASHBOARD_PORT}",
    ],
    "dns": DOCKER_DNS,
    "deploy": {"resources": {"limits": {"cpus": node_cpus, "memory": node_memory}}},
    "depends_on": {
        OPENSEARCH_NODE_NAMES[j]: {"condition": "service_healthy"}
        for j in range(OPENSEARCH_TOTAL_NODES)
    },
}


opensearch_compose_yaml_path = os.path.join(
    LOCALHOST_WORKDIR, "opensearch-cluster.docker-compose.yml"
)
opensearch_compose_yaml_contents = yaml.dump(
    opensearch_compose_dict, default_flow_style=False, sort_keys=False, indent=4
)

with open(opensearch_compose_yaml_path, "w") as f:
    f.write(opensearch_compose_yaml_contents)

print(f"Successfully created: '{os.path.relpath(opensearch_compose_yaml_path)}'")
display(Markdown(f"```yaml\n{opensearch_compose_yaml_contents}\n```"))

Successfully created: 'opensearch-cluster.docker-compose.yml'


```yaml
name: opensearch-cluster
services:
    opensearch-node-1:
        image: opensearchproject/opensearch:3.4.0
        container_name: opensearch-node-1
        environment:
        - cluster.name=opensearch-cluster.mavasbel.vpn.itam.mx
        - node.name=opensearch-node-1
        - discovery.seed_hosts=opensearch-node-1,opensearch-node-2,opensearch-node-3
        - cluster.initial_cluster_manager_nodes=opensearch-node-1,opensearch-node-2,opensearch-node-3
        - http.port=9201
        - bootstrap.memory_lock=true
        - OPENSEARCH_JAVA_OPTS=-Xms1G -Xmx1G
        - OPENSEARCH_INITIAL_ADMIN_PASSWORD=OpenSearchP455
        - OPENSEARCH_HOSTS=["https://opensearch-node-1.mavasbel.vpn.itam.mx:9201","https://opensearch-node-2.mavasbel.vpn.itam.mx:9202","https://opensearch-node-3.mavasbel.vpn.itam.mx:9203"]
        ulimits:
            memlock:
                soft: -1
                hard: -1
            nofile:
                soft: 65536
                hard: 65536
        volumes:
        - .\mount\opensearch-node-1\data:/usr/share/opensearch/data
        - .\mount\opensearch-node-1\performance-analyzer.properties:/usr/share/opensearch/config/opensearch-performance-analyzer/performance-analyzer.properties
        networks:
        - opensearch-cluster
        hostname: opensearch-node-1.mavasbel.vpn.itam.mx
        ports:
        - 9201:9201
        - 16281:16281
        extra_hosts:
        - host.docker.internal:host-gateway
        dns: &id001
        - 10.15.20.1
        deploy:
            resources:
                limits:
                    cpus: '2.0'
                    memory: 2G
        depends_on: {}
        healthcheck:
            test:
            - CMD-SHELL
            - curl -s -k -u admin:${OPENSEARCH_INITIAL_ADMIN_PASSWORD} https://localhost:9201/_cluster/health
                | grep -qv '"status":"red"'
            interval: 10s
            timeout: 10s
            retries: 10
            start_period: 30s
    opensearch-node-2:
        image: opensearchproject/opensearch:3.4.0
        container_name: opensearch-node-2
        environment:
        - cluster.name=opensearch-cluster.mavasbel.vpn.itam.mx
        - node.name=opensearch-node-2
        - discovery.seed_hosts=opensearch-node-1,opensearch-node-2,opensearch-node-3
        - cluster.initial_cluster_manager_nodes=opensearch-node-1,opensearch-node-2,opensearch-node-3
        - http.port=9202
        - bootstrap.memory_lock=true
        - OPENSEARCH_JAVA_OPTS=-Xms1G -Xmx1G
        - OPENSEARCH_INITIAL_ADMIN_PASSWORD=OpenSearchP455
        - OPENSEARCH_HOSTS=["https://opensearch-node-1.mavasbel.vpn.itam.mx:9201","https://opensearch-node-2.mavasbel.vpn.itam.mx:9202","https://opensearch-node-3.mavasbel.vpn.itam.mx:9203"]
        ulimits:
            memlock:
                soft: -1
                hard: -1
            nofile:
                soft: 65536
                hard: 65536
        volumes:
        - .\mount\opensearch-node-2\data:/usr/share/opensearch/data
        - .\mount\opensearch-node-2\performance-analyzer.properties:/usr/share/opensearch/config/opensearch-performance-analyzer/performance-analyzer.properties
        networks:
        - opensearch-cluster
        hostname: opensearch-node-2.mavasbel.vpn.itam.mx
        ports:
        - 9202:9202
        - 16282:16282
        extra_hosts:
        - host.docker.internal:host-gateway
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '2.0'
                    memory: 2G
        depends_on:
            opensearch-node-1:
                condition: service_started
        healthcheck:
            test:
            - CMD-SHELL
            - curl -s -k -u admin:${OPENSEARCH_INITIAL_ADMIN_PASSWORD} https://localhost:9202/_cluster/health
                | grep -qv '"status":"red"'
            interval: 10s
            timeout: 10s
            retries: 10
            start_period: 30s
    opensearch-node-3:
        image: opensearchproject/opensearch:3.4.0
        container_name: opensearch-node-3
        environment:
        - cluster.name=opensearch-cluster.mavasbel.vpn.itam.mx
        - node.name=opensearch-node-3
        - discovery.seed_hosts=opensearch-node-1,opensearch-node-2,opensearch-node-3
        - cluster.initial_cluster_manager_nodes=opensearch-node-1,opensearch-node-2,opensearch-node-3
        - http.port=9203
        - bootstrap.memory_lock=true
        - OPENSEARCH_JAVA_OPTS=-Xms1G -Xmx1G
        - OPENSEARCH_INITIAL_ADMIN_PASSWORD=OpenSearchP455
        - OPENSEARCH_HOSTS=["https://opensearch-node-1.mavasbel.vpn.itam.mx:9201","https://opensearch-node-2.mavasbel.vpn.itam.mx:9202","https://opensearch-node-3.mavasbel.vpn.itam.mx:9203"]
        ulimits:
            memlock:
                soft: -1
                hard: -1
            nofile:
                soft: 65536
                hard: 65536
        volumes:
        - .\mount\opensearch-node-3\data:/usr/share/opensearch/data
        - .\mount\opensearch-node-3\performance-analyzer.properties:/usr/share/opensearch/config/opensearch-performance-analyzer/performance-analyzer.properties
        networks:
        - opensearch-cluster
        hostname: opensearch-node-3.mavasbel.vpn.itam.mx
        ports:
        - 9203:9203
        - 16283:16283
        extra_hosts:
        - host.docker.internal:host-gateway
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '2.0'
                    memory: 2G
        depends_on:
            opensearch-node-1:
                condition: service_started
            opensearch-node-2:
                condition: service_started
        healthcheck:
            test:
            - CMD-SHELL
            - curl -s -k -u admin:${OPENSEARCH_INITIAL_ADMIN_PASSWORD} https://localhost:9203/_cluster/health
                | grep -qv '"status":"red"'
            interval: 10s
            timeout: 10s
            retries: 10
            start_period: 30s
    opensearch-dashboards:
        image: opensearchproject/opensearch-dashboards:3.4.0
        container_name: opensearch-dashboards
        environment:
        - OPENSEARCH_HOSTS=["https://opensearch-node-1.mavasbel.vpn.itam.mx:9201","https://opensearch-node-2.mavasbel.vpn.itam.mx:9202","https://opensearch-node-3.mavasbel.vpn.itam.mx:9203"]
        - OPENSEARCH_JAVA_OPTS=-Xms1G -Xmx1G
        networks:
        - opensearch-cluster
        ports:
        - 5601:5601
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '2.0'
                    memory: 2G
        depends_on:
            opensearch-node-1:
                condition: service_healthy
            opensearch-node-2:
                condition: service_healthy
            opensearch-node-3:
                condition: service_healthy
networks:
    opensearch-cluster:
        driver: bridge

```

In [4]:
!docker compose -f opensearch-cluster.docker-compose.yml up -d --wait

time="2026-01-11T21:18:59-06:00" level=warning msg="The \"OPENSEARCH_INITIAL_ADMIN_PASSWORD\" variable is not set. Defaulting to a blank string."
time="2026-01-11T21:18:59-06:00" level=warning msg="The \"OPENSEARCH_INITIAL_ADMIN_PASSWORD\" variable is not set. Defaulting to a blank string."
time="2026-01-11T21:18:59-06:00" level=warning msg="The \"OPENSEARCH_INITIAL_ADMIN_PASSWORD\" variable is not set. Defaulting to a blank string."
 Container opensearch-node-1  Running
 Container opensearch-node-2  Running
 Container opensearch-node-3  Running
 Container opensearch-dashboards  Recreate
 Container opensearch-dashboards  Recreated
 Container opensearch-node-2  Waiting
 Container opensearch-node-3  Waiting
 Container opensearch-node-1  Waiting
 Container opensearch-node-2  Healthy
 Container opensearch-node-3  Healthy
 Container opensearch-node-1  Healthy
 Container opensearch-dashboards  Starting
 Container opensearch-dashboards  Started
 Container opensearch-node-3  Waiting
 Container